In [6]:
a = [0,0,0,0,0]
n = 7
a = [ 0 for i in range(n)]
a
a = [ [0 for i in range(n)]] # 2차원 배열로 보면 원소 개수가 1개이다.
a
len(a) # 이건 값이 1
len(a[0])

7

In [10]:
n=5
a = [ [0 for i in range(n)] for j in range(n)]
a
#len(a[0])

[[0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0]]

In [11]:
# 더 쉬운 방법
n=5
a = [ [0]*n for j in range(n)]
a

[[0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0]]

In [13]:
#알고리즘적인 문제
n=5
a = [ [0]*n for j in range(n)]
row = 0 # 열
col = -1 # 행
count = 1 #1부터 증가하니까

for i in range(n):
    col += 1
    a[row][col] = count
    count += 1
a

[[1, 2, 3, 4, 5],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0]]

In [15]:
n=5
a = [ [0]*n for j in range(n)]
row = 0 # 열
col = -1 # 행
count = 1 #1부터 증가하니까
num = n # 회전수 담당 변수

for i in range(num):
    col += 1
    a[row][col] = count
    count += 1
    
num -= 1

for i in range(num):
    row += 1
    a[row][col] = count
    count += 1

a

[[1, 2, 3, 4, 5],
 [0, 0, 0, 0, 6],
 [0, 0, 0, 0, 7],
 [0, 0, 0, 0, 8],
 [0, 0, 0, 0, 9]]

In [28]:
def display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()

n=11
a = [ [0]*n for j in range(n)]
row = 0 # 열
col = -1 # 행
count = 1 #1부터 증가하니까
num = n # 회전수 담당 변수


while True:
    for i in range(num):
        col += 1
        a[row][col] = count
        count += 1

    num -= 1 #회전수 감소할 때 소환
    if num == 0:
        break
        
    for i in range(num):
        row += 1 # 행건드릴때
        a[row][col] = count
        count += 1

    for i in range(num):
        col -= 1 #열 건드릴때
        a[row][col] = count
        count += 1

    num -= 1
    if num == 0:
        break
        
    for i in range(num):
        row -= 1
        a[row][col] = count
        count += 1

display(a)

   1   2   3   4   5   6   7   8   9  10  11
  40  41  42  43  44  45  46  47  48  49  12
  39  72  73  74  75  76  77  78  79  50  13
  38  71  96  97  98  99 100 101  80  51  14
  37  70  95 112 113 114 115 102  81  52  15
  36  69  94 111 120 121 116 103  82  53  16
  35  68  93 110 119 118 117 104  83  54  17
  34  67  92 109 108 107 106 105  84  55  18
  33  66  91  90  89  88  87  86  85  56  19
  32  65  64  63  62  61  60  59  58  57  20
  31  30  29  28  27  26  25  24  23  22  21


In [32]:
def display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()

n=5
a = [ [0]*n for j in range(n)]
row = 0 # 열
col = -1 # 행
count = 1 #1부터 증가하니까
num = n # 회전수 담당 변수
sign = 1

while True:
    for i in range(num):
        col += sign #sign 반전코드 만드는것.
        a[row][col] = count
        count += 1

    num -= 1
    if num == 0:
        break
        
    for i in range(num):
        row += sign # 행건드릴때
        a[row][col] = count
        count += 1
    sign = - sign #sign 부호반전이 while 루프 돌면서 발생

display(a)

   1   2   3   4   5
  16  17  18  19   6
  15  24  25  20   7
  14  23  22  21   8
  13  12  11  10   9


In [39]:
def display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()
        

def init(n):
    a = [ [0]*n for j in range(n)]
    return a

def fill(a, n):
    count = 1
    for i in range(n):
        for j in range(n):
            a[i][j] = count
            count += 1
    
    
n=5
a = init(n)
fill(a,n)
display(a)

   1   2   3   4   5
   6   7   8   9  10
  11  12  13  14  15
  16  17  18  19  20
  21  22  23  24  25


In [41]:
def display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()
        

def init(n):
    a = [ [0]*n for j in range(n)]
    return a

def fill(a, n):
    count = 1
    for i in range(n):
        for j in range(n-i):
            a[i+j][i] = count
            count += 1
    
    
n=5
a = init(n)
fill(a,n)
display(a)

   1   0   0   0   0
   2   6   0   0   0
   3   7  10   0   0
   4   8  11  13   0
   5   9  12  14  15


In [45]:
def display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()
        

def init(n):
    a = [ [0]*n for j in range(n)]
    return a

def fill(a, n):
    count = 1
    for i in range(n):
        for j in range(n-i):
            a[i][n-i-1-j] = count
            count += 1
    
    
n=5
a = init(n)
fill(a,n)
display(a)

   5   4   3   2   1
   9   8   7   6   0
  12  11  10   0   0
  14  13   0   0   0
  15   0   0   0   0


In [54]:
def display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()
        

def init(n):
    a = [ [0]*n for j in range(n)]
    return a

def fill(a, n):
    count = 1
    for k in range(2*n-1):
        for i in range(n):
            for j in range(n):
                if (i+j) == k:
                    a[i][j] = count
                    count += 1
    
    
n=5
a = init(n)
fill(a,n)
display(a)

   1   2   4   7  11
   3   5   8  12  16
   6   9  13  17  20
  10  14  18  21  23
  15  19  22  24  25


In [56]:
def display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()
        

def init(n):
    a = [ [0]*n for j in range(n)]
    return a

def fill(a, n):
    count = 1
    for k in range(2*n-1):
        for i in range(n):
            for j in range(n):
                if (i+j) == k:
                    a[j][i] = count
                    count += 1
    
    
n=5
a = init(n)
fill(a,n)
display(a)

   1   3   6  10  15
   2   5   9  14  19
   4   8  13  18  22
   7  12  17  21  24
  11  16  20  23  25


In [67]:
def display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()
        

def init(n):
    a = [ [0]*n for j in range(n)]
    return a

def fill(a, n):
    count = 1
    inc = 0
    flag = 1
    for i in range(n):
        for j in range(n-inc*2):
            a[inc+j][i] = count
            count += 1
        if n//2 == i:
            flag = -flag
        inc += flag
    
n=5
a = init(n)
fill(a,n)
display(a)

   1   0   0   0  13
   2   6   0  10  14
   3   7   9  11  15
   4   8   0  12  16
   5   0   0   0  17


## 클래스 만들기 

In [69]:
class FourCal:
    def add(self, a, b):
        result = a + b
        return result

a = FourCal()
print(a.add(3,4))

7


In [72]:
class FourCal:
    def setdata(self, first, second):
        self.first = first
        self.second = second
    
    def add(self):
        result = self.first + self.second
        return result

a = FourCal()
a.setdata(3,4)
print(a.add())

7


In [80]:
class FourCal:
    def __init__(self, first, second):#생성자
        print("FourCal.__init__()")
        self.first = first
        self.second = second
    
    def setdata(self, first, second):
        self.first = first
        self.second = second
    
    def add(self):
        result = self.first + self.second
        return result

a = FourCal(3,4)
print(a.add())

FourCal.__init__()
7


In [87]:
class MoreFourCal(FourCal):
    def pow(self):
        result = self.first ** self.second
        return result

a = MoreFourCal(3,4)
a.pow()

FourCal.__init__()


81

In [89]:
class FourCal:
    def __init__(self, first, second):#생성자
        print("FourCal.__init__()")
        self.first = first
        self.second = second
    
    def setdata(self, first, second):
        self.first = first
        self.second = second
    
    def add(self):
        result = self.first + self.second
        return result
    
    def div(self):
        result = self.first/self.second
        return result

a = FourCal(3,4)
print(a.div())

FourCal.__init__()
0.75


In [95]:
class FourCal:
    def __init__(self, first, second):#생성자
        print("FourCal.__init__()")
        self.first = first
        self.second = second
    
    def setdata(self, first, second):
        self.first = first
        self.second = second
    
    def add(self):
        result = self.first + self.second
        return result
    
    def div(self):
        print("FourCal.div()")
        result = self.first/self.second
        return result

a = FourCal(3,0)
print(a.div())

FourCal.__init__()


ZeroDivisionError: division by zero

In [96]:
class MoreFourCal(FourCal):
    def pow(self):
        result = self.first ** self.second
        return result
    
    def div(self): #오버라이딩. 부모랑 똑같은 이름의 함수 호출하면 자식의 함수가 호출됨
        print("MoreFourCal.div()")
        if(self.second == 0):
            print('분모는 0으로 지정하면 안되요')
            return
        
        result = self.first/self.second
        return result

a = MoreFourCal(3,0)
a.div()

FourCal.__init__()
MoreFourCal.div()
분모는 0으로 지정하면 안되요


In [107]:
def foo():
    print('foo()')
    
def foo(a):
    print('foo(a)')
    
foo(a)
foo(3) #이건 오버로딩인데 파이썬에서는 지원하지 않는다. 전통적인 객체지향 프로그램은 오버로딩 지원함.
#호출안하면 에러 안난다. 
#위에거 호출하면 에러난다. 
#foo 가 변수같은 존재
#함수도 파이썬에서는 객체라 최신의 함수만 남아 작동하게 된다.

foo(a)
foo(a)


### 클래스 변수 

In [108]:
class Family:
    lastname = '김'#지역변수와 멤버변수 사이에 있는 희안한 존재, 클래스안에만 있다. 클래스 변수. 
                    #멤버변수가 아니다. 파이썬은 클래스도 객체다.
    def foo():
        name = '홍길동' #local 변수
        
print(Family.lastname)

김


In [109]:
a = Family()
print(a.lastname)

김


In [110]:
b = Family()
print(b.lastname)

김


In [111]:
Family.lastname = '박' # 클래스 전체에 소속된 변수이기 때문에 수정하면 통으로 수정됨

In [112]:
print(a.lastname)

박


In [113]:
print(b.lastname)

박


In [114]:
id(Family.lastname)

2378593582336

In [115]:
id(a.lastname)

2378593582336

In [117]:
id(b.lastname)

2378593582336